In [44]:
#Importing libraries
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [58]:
# Insert the names (or paths) of the tsv files
files = [
    "bombina/bombina_corset_DEGS__unk_not-unk.fasta.transdecoder.pep_nr.tsv",
    "bombina/bombina_corset_DEGS__unk_not-unk.fasta.transdecoder.pep_tr.tsv",
    "bombina/bombina_corset_DEGS__unk_not-unk.fasta.transdecoder.pep_sp.tsv"   
]

# Insert the titles of the graph
title = "bombina_pachypus_blastp"

# Insert the databases names
databases_names =[
    "Nr", 
    "TrEMBL",
    "Swiss-Prot",
]

table_path = "./bombina/bombina_unref_vs_not_unkref_table_padj_0.05----log2fc_1 (4).tsv"

# Insert the path of the graphs
path = "bombina/report/" + title

# Set the outformat
# e.g. 
# outfmt = "qseqid qlen sseqid sallseqid slen qstart qend sstart send qseq full_qseq sseq full_sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos qframe btop cigar staxids sscinames sskingdoms skingdoms sphylums stitle salltitles qcovhsp scovhsp qtitle qqual full_qqual qstrand"
# If there are column names in the file then set outfmt = None
outfmt = "qseqid qlen sseqid sallseqid slen qstart qend sstart send qseq full_qseq sseq full_sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos qframe btop cigar staxids sscinames sskingdoms skingdoms sphylums stitle salltitles qcovhsp scovhsp qtitle qqual full_qqual qstrand"

# Columns names
features = ["transcript", "row", "log2foldchange", "padj", 
            "protein_accession", "sequence_identity", "alignment_length", 
            "evalue", "database", "locus_name", "sequence_description",
            "sequence_length", "organism", "protein_product"]

df = pd.DataFrame(columns=features)

table = pd.read_csv(table_path, sep='\t')

In [62]:
for f in files:

    #Import the dataset
    df_tmp = pd.read_csv(f, sep="\t", names=outfmt.split())

    df_tmp['transcript'] = df_tmp.qseqid
    df_tmp['title'] = title
    df_tmp['sequence_identity'] = df_tmp.pident
    df_tmp['alignment_length'] = df_tmp.length
    df_tmp['evalue'] = df_tmp.evalue
    df_tmp['sequence_description'] = df_tmp.stitle
    df_tmp['sequence_length'] = df_tmp.slen

    if "OS=" not in df_tmp.stitle[0]:    
        def get_sciname(x):
            
            os_index = - x[::-1].index('[')

            return x[os_index:-1]

        # Useful functions
        def get_protein_function(x):

            x_l = x.split(" ")

            return ' '.join(x_l[1:x_l.index(next(x for x in x_l if x.startswith('[')))])
        
        def get_protein_accession(x):
            return x.split(" ")[0]
        
        def get_locus_name(x):
            return None
    else:
        def get_sciname(x):

            os_index = x.index('OS=')
            ox_index = x.index('OX=')

            return x[os_index+3:ox_index-1]

        # Useful functions
        def get_protein_function(x):

            x_l = x.split(" ")

            return ' '.join(x_l[1:x_l.index(next(x for x in x_l if x.startswith('OS=')))])
        
        def get_protein_accession(x):
            return x.split("|")[1]
        
        def get_locus_name(x):
            return x.split("|")[2]
    
    df_tmp['organism'] = df_tmp.stitle.apply(lambda x: get_sciname(x))
    df_tmp['protein_accession'] = df_tmp.sseqid.apply(lambda x: get_protein_accession(x))
    df_tmp['protein_product'] = df_tmp.stitle.apply(lambda x: get_protein_function(x))
    df_tmp['locus_name'] = df_tmp.sseqid.apply(lambda x: get_locus_name(x))

    #pd.concat([])

In [63]:
df_tmp[["protein_accession", "sequence_identity", "alignment_length"]]

,protein_accession,sequence_identity,alignment_length
0,P46058,34.5,328
1,Q91321,28.0,157
2,P48649,28.4,155
3,Q91320,28.0,157
4,P02522,28.8,139
...,...,...,...
1483,P30028,27.1,170
1484,P0C691,27.1,170
1485,P17192,27.6,170
1486,Q6NS60,90.8,119


In [29]:
df_sp = pd.read_csv(files[2], sep="\t", names=outfmt.split())
df_tr = pd.read_csv(files[1], sep="\t", names=outfmt.split())
df_nr = pd.read_csv(files[0], sep="\t", names=outfmt.split())

In [16]:
df_sp.stitle[0]

'sp|P46058|EDSP_CYNPY Epidermal differentiation-specific protein OS=Cynops pyrrhogaster OX=8330 PE=2 SV=1'

In [1]:
#Creating a dictonary list where each dict contains all sequences hits count
sets_list = []

for f in files:
    #Import the dataset
    df = pd.read_csv(f, sep="\t", names=outfmt.split())
    
    #Drop Nan rows
    df.dropna(subset=[index], inplace=True)
    
    #Create and append the dict
    sets_list.append(set(df[index]) - {index})

NameError: name 'files' is not defined